In [94]:
import boto3
import pandas as pd
from io import StringIO,BytesIO# bytesio because we are saving in s3 in 
#parquet format whichnwill be usefull in orchastrationa nd airflow
from datetime import datetime,timedelta

# Adapter Layer

In [95]:
# Adapter Layer

def read_csv_to_df(bucket, key, decoding = 'utf-8', sep = ','):
    csv_obj = bucket.Object(key=key).get().get('Body').read().decode(decoding)
    data = StringIO(csv_obj)
    df = pd.read_csv(data, delimiter=sep)
    return df

def write_df_to_s3(bucket, df, key):
    out_buffer = BytesIO()
    df.to_parquet(out_buffer, index=False)
    bucket.put_object(Body=out_buffer.getvalue(), Key=key)
    return True

def write_df_to_s3_csv(bucket, df, key):
    out_buffer = StringIO()
    df.to_csv(out_buffer, index=False)
    bucket.put_object(Body=out_buffer.getvalue(), Key=key)
    return True

def list_files_in_prefix(bucket, prefix):
    files = [obj.key for obj in bucket.objects.filter(Prefix=prefix)]
    return files

# Application layer

In [96]:
# Application Layer

def extract(bucket, date_list):
    files = [key for date in date_list for key in list_files_in_prefix(bucket, date)]
    df = pd.concat([read_csv_to_df(bucket, obj) for obj in files], ignore_index=True)
    return df

def transform_report1(df, columns, arg_date):
    df = df.loc[:, columns]
    df.dropna(inplace=True)
    df['opening_price'] = df.sort_values(by=['Time']).groupby(['ISIN', 'Date'])['StartPrice'].transform('first')
    df['closing_price'] = df.sort_values(by=['Time']).groupby(['ISIN', 'Date'])['StartPrice'].transform('last')
    df = df.groupby(['ISIN', 'Date'], as_index=False).agg(opening_price_eur=('opening_price', 'min'), closing_price_eur=('closing_price', 'min'), minimum_price_eur=('MinPrice', 'min'), maximum_price_eur=('MaxPrice', 'max'), daily_traded_volume=('TradedVolume', 'sum'))
    df['prev_closing_price'] = df.sort_values(by=['Date']).groupby(['ISIN'])['closing_price_eur'].shift(1)
    df['change_prev_closing_%'] = (df['closing_price_eur'] - df['prev_closing_price']) / df['prev_closing_price'] * 100
    df.drop(columns=['prev_closing_price'], inplace=True)
    df = df.round(decimals=2)
    df = df[df.Date >= arg_date]
    return df

def load(bucket, df, trg_key, trg_format,meta_key, extract_date_list):
    key = trg_key + datetime.today().strftime("%Y%m%d_%H%M%S") + trg_format
    write_df_to_s3(bucket, df, key)
    update_meta_file(bucket, meta_key, extract_date_list)
    return True

def etl_report1(src_bucket, trg_bucket, date_list, columns, arg_date, trg_key, trg_format,meta_key):
    df = extract(src_bucket, date_list)
    df = transform_report1(df, columns, arg_date)
    extract_date_list = [date for date in date_list if date >= arg_date]
    load(trg_bucket, df, trg_key, trg_format,meta_key, extract_date_list)
    return True
    

In [97]:
def return_date_list(bucket, arg_date, src_format, meta_key):
    min_date = datetime.strptime(arg_date, src_format).date() - timedelta(days=1)
    today = datetime.today().date()
    try:
        df_meta = read_csv_to_df(bucket, meta_key)
        dates = [(min_date + timedelta(days=x)) for x in range(0, (today-min_date).days + 1)]
        src_dates = set(pd.to_datetime(df_meta['source_date']).dt.date)
        
        # Include the date immediately after the last available date in df_meta
        if src_dates:
            last_date_in_meta = max(src_dates)
            min_date = last_date_in_meta
        else:
            min_date = min_date

        dates_missing = set(dates) - src_dates

        if dates_missing:
            min_date = min(dates_missing)
            return_dates = [date.strftime(src_format) for date in dates if date >= min_date]
            return_min_date = min_date.strftime(src_format)
        else:
            return_dates = []
            return_min_date = datetime(2200, 1, 1).strftime(src_format)
    except bucket.meta.client.exceptions.NoSuchKey:
        return_dates = [(min_date + timedelta(days=x)).strftime(src_format) for x in range(0, (today-min_date).days + 1)]
        return_min_date = arg_date
    return return_min_date, return_dates


In [98]:
'''# Application Layer - not core

def return_date_list(bucket, arg_date, src_format, meta_key):
    min_date = datetime.strptime(arg_date, src_format).date() - timedelta(days=1)
    today = datetime.today().date()
    try:
        df_meta = read_csv_to_df(bucket, meta_key)
        dates = [(min_date + timedelta(days=x)) for x in range(0, (today-min_date).days + 1)]
        src_dates = set(pd.to_datetime(df_meta['source_date']).dt.date)
        dates_missing = set(dates[1:]) - src_dates
        if dates_missing:
            min_date = min(set(dates[1:]) - src_dates) - timedelta(days=1)
            return_dates = [date.strftime(src_format) for date in dates if date >= min_date]
            return_min_date = (min_date + timedelta(days=1)).strftime(src_format)
        else:
            return_dates = []
            return_min_date = datetime(2200, 1, 1).date()
    except bucket.meta.client.exceptions.NoSuchKey:
        return_dates = [(min_date + timedelta(days=x)).strftime(src_format) for x in range(0, (today-min_date).days + 1)]
        return_min_date = arg_date
    return return_min_date, return_dates

def update_meta_file(bucket, meta_key, extract_date_list):
    df_new = pd.DataFrame(columns=['source_date', 'datetime_of_processing'])
    df_new['source_date'] = extract_date_list
    df_new['datetime_of_processing'] = datetime.today().strftime('%Y-%m-%d')
    df_old = read_csv_to_df(bucket, meta_key)
    df_all = pd.concat([df_old, df_new])
    write_df_to_s3_csv(bucket, df_all, meta_key)'''

"# Application Layer - not core\n\ndef return_date_list(bucket, arg_date, src_format, meta_key):\n    min_date = datetime.strptime(arg_date, src_format).date() - timedelta(days=1)\n    today = datetime.today().date()\n    try:\n        df_meta = read_csv_to_df(bucket, meta_key)\n        dates = [(min_date + timedelta(days=x)) for x in range(0, (today-min_date).days + 1)]\n        src_dates = set(pd.to_datetime(df_meta['source_date']).dt.date)\n        dates_missing = set(dates[1:]) - src_dates\n        if dates_missing:\n            min_date = min(set(dates[1:]) - src_dates) - timedelta(days=1)\n            return_dates = [date.strftime(src_format) for date in dates if date >= min_date]\n            return_min_date = (min_date + timedelta(days=1)).strftime(src_format)\n        else:\n            return_dates = []\n            return_min_date = datetime(2200, 1, 1).date()\n    except bucket.meta.client.exceptions.NoSuchKey:\n        return_dates = [(min_date + timedelta(days=x)).strftim

In [99]:
# main function entrypoint

def main():
    # Parameters/Configurations
    # Later read config
    arg_date='2022-12-30'
    src_format = '%Y-%m-%d'
    src_bucket = 'xetra-1234'
    trg_bucket = 'xetra-sm'
    columns = ['ISIN', 'Date', 'Time', 'StartPrice', 'MaxPrice', 'MinPrice', 'EndPrice', 'TradedVolume']
    trg_key = 'xetra_smitesh_report'
    trg_format = '.parquet'
    meta_key = 'meta_file.csv'
    
    # Init
    s3 = boto3.resource('s3')
    bucket_src = s3.Bucket(src_bucket)
    bucket_trg = s3.Bucket(trg_bucket)
    
    # run application
    extract_date, date_list = return_date_list(bucket_trg, arg_date, src_format, meta_key)
    etl_report1(bucket_src, bucket_trg, date_list, columns, extract_date, trg_key, trg_format,meta_key)

In [100]:
# run

main()

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_4716\2579682883.py:7: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  src_dates = set(pd.to_datetime(df_meta['source_date']).dt.date)
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_4716\1485487011.py:5: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([read_csv_to_df(bucket, obj) for obj in files], ignore_index=True)


## Reading the uploaded file

In [101]:
trg_bucket = 'xetra-sm'
s3 = boto3.resource('s3')
bucket_trg = s3.Bucket(trg_bucket)
for obj in bucket_trg.objects.all():
    print(obj.key)

meta_file.csv
xetra_smitesh_report20240821_210632.parquet
xetra_smitesh_report20240821_212312.parquet
xetra_smitesh_report20240901_165708.parquet
xetra_smitesh_report20240901_171909.parquet
xetra_smitesh_report20240901_195045.parquet
xetra_smitesh_report20240901_204213.parquet
xetra_smitesh_report20240901_205024.parquet
xetra_smitesh_report20240901_221323.parquet
xetra_smitesh_report20240901_223928.parquet
xetra_smitesh_report20240901_225542.parquet


In [104]:
prq_obj = bucket_trg.Object(key='xetra_smitesh_report20240901_225542.parquet').get().get('Body').read()
data = BytesIO(prq_obj)
df_report = pd.read_parquet(data)

In [105]:
df_report

,ISIN,Date,opening_price_eur,closing_price_eur,minimum_price_eur,maximum_price_eur,daily_traded_volume,change_prev_closing_%
0,AT000000STR1,2022-12-29,36.60,36.70,35.75,36.70,1773,NaN
1,AT000000STR1,2022-12-30,36.60,36.70,35.75,36.70,1773,0.0
2,AT000000STR1,2022-12-31,36.60,36.70,35.75,36.70,1773,0.0
3,AT00000FACC2,2022-12-29,8.05,8.57,7.87,8.57,10205,NaN
4,AT00000FACC2,2022-12-30,8.05,8.57,7.87,8.57,10205,0.0
...,...,...,...,...,...,...,...,...
9691,XS2376095068,2022-12-30,34.29,36.50,34.06,36.50,1000,0.0
9692,XS2376095068,2022-12-31,34.29,36.50,34.06,36.50,1000,0.0
9693,XS2434891219,2022-12-29,3.44,3.66,3.42,3.66,0,NaN
9694,XS2434891219,2022-12-30,3.44,3.66,3.42,3.66,0,0.0
